In [273]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [274]:
import scipy.stats as st
from functions import *
from graphics import *
from statistics_tests import *
import pandas as pd

df_final_demo = get_final_demo_df()

df_web_data_concat = get_web_data_df()

df_final_exp = get_final_exp_df()

df_all = get_df_all()

LIMPIEZA DE DATOS

In [275]:
df_all = df_all.drop("clnt_tenure_yr", axis=1)      # We don´t need this column because is derivative from clnt_tenure_mnth

QUITAMOS OUTLIERS DEL DF_ALL Y DEL QUE NO TIENE DUPLICADOS

In [276]:
df_all = outlier_slayer(df_all)

In [277]:
df_all.sort_values(by=["client_id", "visit_id", "date_time"], ascending=[True, True, False], inplace=True)

In [278]:
# df_all["time_diff"] = df_all.groupby(["client_id", "visit_id"])["date_time"].shift(1) - df_all["date_time"]

In [279]:
df_all["time_diff"] = abs(df_all.groupby(["client_id", "visit_id"])["date_time"].diff().shift(-1).fillna(pd.Timedelta(seconds=0)) ).dt.total_seconds()

In [280]:
df_inter = df_all.copy()
df_inter.drop_duplicates(subset=["visit_id", "process_step"], keep="last", inplace=True)

In [281]:
df_all["time_diff_cleaned"] = abs(df_inter.groupby(["client_id", "visit_id"])["date_time"].diff().shift(-1).fillna(pd.Timedelta(seconds=0)) ).dt.total_seconds()

In [282]:
df_all["repetition"] = df_all.duplicated(subset=["visit_id", "process_step"], keep="last").astype(int)          #0 = no, 1 = yes

In [283]:
df_all["last_step"] = df_all.groupby("visit_id")["process_step"].transform("max")

In [284]:
df_all.head()

,client_id,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,visitor_id,visit_id,process_step,date_time,variation,time_diff,time_diff_cleaned,repetition,last_step
255392,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,4,2017-04-15 13:00:34,1,20.0,20.0,0,4
255393,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,3,2017-04-15 13:00:14,1,99.0,99.0,0,4
255394,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,2,2017-04-15 12:58:35,1,32.0,32.0,0,4
255395,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,1,2017-04-15 12:58:03,1,7.0,7.0,0,4
255396,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,0,2017-04-15 12:57:56,1,0.0,0.0,0,4


In [285]:
# count = df_all[(df_all["process_step"] == 4) & 
#                 (df_all["repetition"] == 0) & 
#                 (df_all["variation"] == 1)].shape[0]

# count

In [286]:
df_all["time_diff_cleaned"][df_all["variation"] == 0].isna().sum()

23746

In [287]:
df_all["time_diff_cleaned"][df_all["variation"] == 1].isna().sum()

32859

In [288]:
bins = [0, 25, 40, 60, 100]  # Ajusta los valores de los límites según las necesidades
labels = ['Young (0-25)', 'Young Adult (26-40)', 'Mature Adult (41-60)', 'Senior (60+)']

# Crear la nueva columna 'age_segment' con los tramos de edad
df_all['age_segment'] = pd.cut(df_all['clnt_age'], bins=bins, labels=labels, right=False)

In [289]:
df_all.time_diff_cleaned.notna().sum()

163766

In [290]:
df_all[df_all["variation"] == 1].time_diff.notna().sum() - df_all[df_all["variation"] == 1].time_diff_cleaned.notna().sum()

32859

In [291]:
df_all[df_all["variation"] == 1].time_diff.notna().sum()

122261

In [292]:
df_all.head()

,client_id,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,visitor_id,visit_id,process_step,date_time,variation,time_diff,time_diff_cleaned,repetition,last_step,age_segment
255392,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,4,2017-04-15 13:00:34,1,20.0,20.0,0,4,Young Adult (26-40)
255393,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,3,2017-04-15 13:00:14,1,99.0,99.0,0,4,Young Adult (26-40)
255394,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,2,2017-04-15 12:58:35,1,32.0,32.0,0,4,Young Adult (26-40)
255395,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,1,2017-04-15 12:58:03,1,7.0,7.0,0,4,Young Adult (26-40)
255396,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,0,2017-04-15 12:57:56,1,0.0,0.0,0,4,Young Adult (26-40)


In [293]:
df_all.to_csv("data/df_powerbi.csv")